In [ ]:
import os
import cv2
import numpy as np


parent_dir = '/Users/saikarthik/Desktop/soft computing project /Alzheimer_MRI_4_classes_dataset'


categories = os.listdir(parent_dir)


category_images = {}
category_labels = {}


for category in categories:
    category_dir = os.path.join(parent_dir, category)
    

    if os.path.isdir(category_dir) and category != '.DS_Store':
        all_files = os.listdir(category_dir)
        image_files = [filename for filename in all_files if filename.lower().endswith(('.png', '.jpg', '.jpeg'))]
        images = []  # An empty list to store the processed images for the current category
        labels = [category] * len(image_files)
        

        for image_file in image_files:
            image_path = os.path.join(category_dir, image_file)
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Converting to RGB format
            resized_image = cv2.resize(image_rgb, (224, 224))  # Resizing the image to a 224x224 pixel size
            images.append(resized_image)
    
        category_images[category] = np.array(images)
        category_labels[category] = np.array(labels)

        print(f"Shape of images for category '{category}': {category_images[category].shape}")


In [ ]:
import matplotlib.pyplot as plt


for category in categories:
    if category not in category_images:
        continue
    
    sample_images = category_images[category][:10]  # Taking the first 10 images as a sample

    plt.figure(figsize=(12, 6))
    for i in range(len(sample_images)):
        plt.subplot(2, 5, i + 1)
        plt.imshow(sample_images[i])
        plt.title(f"Image {i+1}")
        plt.axis('off')
    plt.suptitle(f"Category: {category}", fontsize=14)
    plt.show()


# Data Preprocessing


In [ ]:

for category in category_images:
    category_images[category] = category_images[category] / 255.0


min_value = np.min(category_images[category])
max_value = np.max(category_images[category])


print(f"Minimum Pixel Value: {min_value}")
print(f"Maximum Pixel Value: {max_value}")


# Data Augmentation


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,  # Rotating the image by up to 20 degrees
    width_shift_range=0.2,  # Shifting the width by up to 20% of the image width
    height_shift_range=0.2,  # Shifting the height by up to 20% of the image height
    shear_range=0.2, 
    zoom_range=0.2,  # Zoom in by up to 20%
    horizontal_flip=True,  # Fliping the image horizontally
    fill_mode='nearest'  # Fill in new pixels with the nearest existing pixel
)

output_dir = '/Users/saikarthik/Desktop/soft computing project /Alzheimer_MRI_4_classes_dataset'  # Changing to our desired output directory

# Creating the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Looping through each category and apply data augmentation
for category in categories:
    if category not in category_images:
        continue

    images = category_images[category]
    category_output_dir = os.path.join(output_dir, category)
    if not os.path.exists(category_output_dir):
        os.makedirs(category_output_dir)
    i = 0
    for batch in datagen.flow(images, batch_size=1):
        augmented_image = batch[0]
        augmented_image = (augmented_image * 255).astype(np.uint8)  
        filename = f'{i}.jpg' 
        save_path = os.path.join(category_output_dir, filename)
        cv2.imwrite(save_path, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))
        i += 1
        if i >= 100:  
            break


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model


base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# removing the top classification layers
feature_extractor = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_pool').output)


parent_dir = '/Users/saikarthik/Desktop/soft computing project /Alzheimer_MRI_4_classes_dataset'

# List of  all subdirectories 
categories = os.listdir(parent_dir)


category_features = {}
category_labels = {}

# Looping through each subdirectory
for category in categories:
    category_dir = os.path.join(parent_dir, category)
    

    if os.path.isdir(category_dir) and category != '.DS_Store':
        all_files = os.listdir(category_dir)
        
        image_files = [filename for filename in all_files if filename.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        # Initializing lists to store features and labels for the category
        features = []  
        labels = [category] * len(image_files)
        
    
        for image_file in image_files:
            image_path = os.path.join(category_dir, image_file)
            image = cv2.imread(image_path)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
            resized_image = cv2.resize(image_rgb, (224, 224))  
            preprocessed_image = preprocess_input(np.expand_dims(resized_image, axis=0))  
            features.append(feature_extractor.predict(preprocessed_image)[0])  
        

        category_features[category] = np.array(features)
        category_labels[category] = np.array(labels)




In [36]:
# Combine all features and labels into a single dataset
all_features = np.concatenate(list(category_features.values()))
all_labels = np.concatenate(list(category_labels.values()))

In [37]:
x = all_features.reshape(all_features.shape[0], -1)


In [38]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

x = scaler.fit_transform(x)


In [ ]:
x

In [40]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(all_labels)

In [ ]:
y

In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [43]:
from tensorflow.keras.utils import to_categorical
num_classes=4
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
y_train, y_test 

# Model training

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from deap import base, creator, tools
import os
import logging
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from deap import base, creator, tools, algorithms
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

warnings.filterwarnings('ignore')
tf_logger = tf.get_logger()
tf_logger.setLevel(logging.ERROR)  
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'





def create_ann_model(learning_rate=0.01, hidden_units=64):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=x_train.shape[1], activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))  # Output layer for classification
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

ann_model = create_ann_model()
ann_model.fit(x_train, y_train, epochs=50, batch_size=32, validation_split=0.2)


loss, accuracy = ann_model.evaluate(x_test, y_test)
print(f'ANN Accuracy: {accuracy:.2f}')


if 'FitnessMax' not in creator.__dict__:
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))

if 'Individual' not in creator.__dict__:
    creator.create("Individual", list, fitness=creator.FitnessMax)

def evaluate(individual):
    learning_rate = individual[0]
    hidden_units = individual[1]
    
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=x_train.shape[1], activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train, epochs=10, batch_size=32, verbose=0, validation_data=(x_test, y_test_categorical))
    
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    
    y_pred = np.argmax(model.predict(x_test), axis=-1)
    y_true = np.argmax(y_test, axis=-1)
    
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    tf.keras.backend.clear_session()  
    return (accuracy, loss, precision, recall, f1)
toolbox = base.Toolbox()
toolbox.register("learning_rate", np.random.uniform, 0.001, 0.1)
toolbox.register("hidden_units", np.random.randint, 16, 128)
toolbox.register("individual", tools.initCycle, creator.Individual,(toolbox.learning_rate, toolbox.hidden_units), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

best_accuracies = []
losses = []
precisions = []
recalls = []
f1_scores = []

population = toolbox.population(n=20)  
ngen = 20 
for gen in range(ngen):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if np.random.rand() < 0.5:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
            
    for mutant in offspring:
        if np.random.rand() < 0.2:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring

    fits = [ind.fitness.values for ind in population]
    
    best_accuracy, best_loss, best_precision, best_recall, best_f1 = max(fits, key=lambda x: x[0])
    best_accuracies.append(best_accuracy)
    losses.append(best_loss)
    precisions.append(best_precision)
    recalls.append(best_recall)
    f1_scores.append(best_f1)

    print(f"Generation {gen+1}/{ngen}, Best Accuracy: {best_accuracy:.4f}, Best Loss: {best_loss:.4f}, "
          f"Precision: {best_precision:.4f}, Recall: {best_recall:.4f}, F1 Score: {best_f1:.4f}")

# Plot the results
plt.figure(figsize=(12, 5))

# Plot Best Accuracy
plt.subplot(1, 2, 1)
plt.plot(range(1, ngen + 1), best_accuracies, marker='o', linestyle='-', color='b', label='Best Accuracy')
plt.title('Best Accuracy Across Generations')
plt.xlabel('Generation')
plt.ylabel('Accuracy')
plt.xticks(ticks=range(1, ngen + 1))  
plt.legend()
plt.grid()

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(range(1, ngen + 1), losses, marker='o', linestyle='-', color='r', label='Best Loss')
plt.title('Loss Across Generations')
plt.xlabel('Generation')
plt.ylabel('Loss')
plt.xticks(ticks=range(1, ngen + 1))  
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

best_idx = np.argmax([ind.fitness.values[0] for ind in population])
best_ind = population[best_idx]
print("Best Individual: Learning Rate = {:.4f}, Hidden Units = {}".format(best_ind[0], best_ind[1]))
print("Best Fitness: {:.2f}".format(fits[best_idx][0]))

optimized_ann_model = Sequential()
optimized_ann_model.add(Dense(best_ind[1], input_dim=x_train.shape[1], activation='relu'))
optimized_ann_model.add(Dense(num_classes, activation='softmax'))
optimized_ann_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=best_ind[0]), 
                            loss='categorical_crossentropy', 
                            metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_accuracy', 
                               patience=5,  
                               restore_best_weights=True)  

history = optimized_ann_model.fit(x_train, y_train, 
                                   epochs=50, 
                                   batch_size=32, 
                                   validation_split=0.2,
                                   callbacks=[early_stopping])  

loss, accuracy = optimized_ann_model.evaluate(x_test, y_test)
print(f'Optimized ANN Accuracy: {accuracy:.2f}')

y_pred = np.argmax(optimized_ann_model.predict(x_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)

final_precision = precision_score(y_true, y_pred, average='weighted')
final_recall = recall_score(y_true, y_pred, average='weighted')
final_f1 = f1_score(y_true, y_pred, average='weighted')
conf_matrix=confusion_matrix(y_true, y_pred, average='weighted')

print(f'Final Precision: {final_precision:.2f}, Final Recall: {final_recall:.2f}, Final F1 Score: {final_f1:.2f}')

optimized_ann_model.save('optimized_ann_model.h5')
print("Model saved as 'optimized_ann_model.h5'")

In [ ]:
loss, accuracy = optimized_ann_model.evaluate(x_test, y_test)
print(f'Optimized ANN Accuracy: {accuracy:.2f}')

In [ ]:
import numpy as np
import pandas as pd


x_combined = np.concatenate((x_train, x_test), axis=0)

y_combined = np.concatenate((y_train, y_test), axis=0)

y_combined = np.argmax(y_combined, axis=1)

num_features = x_combined.shape[1]  
feature_columns = [f'feature_{i}' for i in range(num_features)]

df_combined = pd.DataFrame(x_combined, columns=feature_columns)

df_combined['label'] = y_combined

print(df_combined.head())

df_combined.to_csv('dataset.csv', index=False)


In [39]:
df=df_combined

In [48]:
from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [ ]:
df.head()